In [1]:
import pandas as  pd
from sqlalchemy import create_engine
import pymysql
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from TickTocTest.ticktoctest.utils import Return_API_response
from TickTocTest.ticktoctest.models import Base, Ada, Btc, Neo, Mana, Bitfinex_DB_Tables, Compare_DB_Tables

In [ ]:
bitfinexURL = 'https://api.bitfinex.com/v2/candles/trade:'
compareURL = 'https://min-api.cryptocompare.com/data/histohour?'

# BITFINEX: 

✌

In [ ]:
resp = Return_API_response()

In [ ]:
data = resp.api_response(bitfinexURL + '6h:tBTCUSD/hist?limit=10')[::-1]
df=  pd.DataFrame(data, columns=['MTS','Open','Close','High','Low','Volume'])
df['MTS'] = pd.to_datetime(df['MTS'], unit='ms')
df.set_index('MTS',drop=True,inplace=True)
df

In [ ]:
df.index[-1]

In [ ]:
resp.close_session()
resp2= Return_API_response()

# CryptoCompare

In [ ]:
data = resp2.api_response(compareURL+ 'fsym=BTC&tsym=USD&limit=10&aggregate=6&e=CCCAGG')
compDF= pd.DataFrame.from_dict(data['Data'])
compDF

In [ ]:
compDF['MTS'] = pd.to_datetime(compDF['time'], unit='s')
compDF['Volume'] = compDF['volumefrom'] + compDF['volumeto']
compDF.drop(['time','volumefrom','volumeto'], inplace=True, axis=1)
compDF.set_index('MTS', drop=True, inplace=True)
compDF.rename(index=str, columns={'close': 'Close', 'open': 'Open', 'low': 'Low', 'high': 'High'}, inplace=True)
compDF = compDF[['Open', 'Close', 'High', 'Low', 'Volume']]
compDF

In [ ]:
resp2.close_session()

## Binance API

Binnance API.  https://api.binance.com
 - All time and timestamp related fields are in milliseconds.
 - HTTP 4XX return codes are used for for malformed requests; the issue is on the sender's side.
 - HTTP 429 return code is used when breaking a request rate limit.
 - HTTP 418 return code is used when an IP has been auto-banned for continuing to send requests after receiving 429 codes.
 - HTTP 5XX return codes are used for internal errors; the issue is on Binance's side. It is important to NOT treat this as a failure operation; the execution status is UNKNOWN and could have been a success.

In [13]:
binance_URL  = 'https://api.binance.com/api/v1/klines?symbol=BCHSVUSDT&interval=1h&limit=1000'
server_time  = 'https://api.binance.com/api/v1/time'
# ticker_price = 'https://api.binance.com/api/v3/ticker/price?symbol=GVTUSDT'

In [14]:
res= Return_API_response()

In [15]:
data = res.api_response(server_time)
data

{'serverTime': 1543382693301}

In [16]:
BiTime = pd.to_datetime(data['serverTime'], unit='ms')
BiTime

Timestamp('2018-11-28 05:24:53.301000')

In [17]:
BiData = res.api_response(binance_URL)
# ticker = res.api_response(ticker_price)
res.close_session()

In [18]:
BiData[0]

[1542355200000,
 '89.74000000',
 '152.00000000',
 '78.00000000',
 '123.85000000',
 '46140.64683000',
 1542358799999,
 '5386433.38638400',
 11388,
 '22998.01173000',
 '2692742.50792170',
 '0']

In [19]:
df = pd.DataFrame(BiData, columns=['MTS','Open','High','Low','Close','Volume', 'MTS_close', 'a', 'b', 'c', 'd','e'])
df.tail()

,MTS,Open,High,Low,Close,Volume,MTS_close,a,b,c,d,e
281,1543366800000,100.04000000,101.10000000,92.00000000,93.24000000,42666.12259000,1543370399999,4087763.14501830,6668,13234.94878000,1270761.84712240,0
282,1543370400000,93.24000000,97.00000000,92.39000000,94.10000000,22141.80863000,1543373999999,2090106.73866700,4746,12216.65625000,1154972.28281680,0
283,1543374000000,94.37000000,96.39000000,93.09000000,93.34000000,13054.41881000,1543377599999,1238258.80145050,3414,6277.00275000,596792.93695240,0
284,1543377600000,93.34000000,95.34000000,91.00000000,92.66000000,15889.17210000,1543381199999,1477786.34358780,3941,7127.83906000,664733.23294630,0
285,1543381200000,92.52000000,93.55000000,91.01000000,91.53000000,5593.50623000,1543384799999,514582.86888860,1475,2578.26885000,237713.31934840,0


In [20]:
df.drop(['MTS_close','a','b','c','d','e'], inplace=True, axis=1)
df['MTS'] = pd.to_datetime(df['MTS'], unit='ms')
df.tail()

,MTS,Open,High,Low,Close,Volume
281,2018-11-28 01:00:00,100.04000000,101.10000000,92.00000000,93.24000000,42666.12259000
282,2018-11-28 02:00:00,93.24000000,97.00000000,92.39000000,94.10000000,22141.80863000
283,2018-11-28 03:00:00,94.37000000,96.39000000,93.09000000,93.34000000,13054.41881000
284,2018-11-28 04:00:00,93.34000000,95.34000000,91.00000000,92.66000000,15889.17210000
285,2018-11-28 05:00:00,92.52000000,93.55000000,91.01000000,91.53000000,5593.50623000


In [21]:
df.set_index('MTS', drop=True, inplace=True)
df = df[['Open', 'Close', 'High', 'Low', 'Volume']]
df.head()

,Open,Close,High,Low,Volume
MTS,,,,,
2018-11-16 08:00:00,89.74000000,123.85000000,152.00000000,78.00000000,46140.64683000
2018-11-16 09:00:00,123.84000000,157.25000000,168.00000000,120.50000000,64086.19721000
2018-11-16 10:00:00,157.99000000,159.00000000,172.99000000,147.00000000,37605.13161000
2018-11-16 11:00:00,159.00000000,144.80000000,159.68000000,136.00000000,29032.53461000
2018-11-16 12:00:00,144.79000000,136.50000000,146.01000000,134.00000000,16072.82751000


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 286 entries, 2018-11-16 08:00:00 to 2018-11-28 05:00:00
Data columns (total 5 columns):
Open      286 non-null object
Close     286 non-null object
High      286 non-null object
Low       286 non-null object
Volume    286 non-null object
dtypes: object(5)
memory usage: 7.8+ KB


In [23]:
df['Close'].tail()

MTS
2018-11-28 01:00:00    93.24000000
2018-11-28 02:00:00    94.10000000
2018-11-28 03:00:00    93.34000000
2018-11-28 04:00:00    92.66000000
2018-11-28 05:00:00    91.53000000
Name: Close, dtype: object

In [ ]:
df['Close'].plot()